In [3]:
import pickle

with open("Dataset/amazon_baby/processed/baby.pkl", 'rb') as file:
    loaded_data = pickle.load(file)


/home/lizhuofeng/.local/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [4]:
longest_review = max(loaded_data.text_edges, key=len)

In [6]:
longest_review

'This review, titled Oh, how I have struggled!, gives the product a rating of 3.0. The reviewer stated, This is a very long review, and I apologize for the length, but hope the amount of detail included will help someone.  I\'ve tried my best to organize it in a way that you can find the parts that suit you.<br /><br />MY EXPERIENCE AS A CLOTH DIAPER USER:<br />I\'ve been using Charlie Banana One Size since the day we came home from the hospital and now that my daughter is 4 months, I\'m ready to share my experience with other shoppers needing guidance.  This is my third daughter, but my first time to cloth diaper.  My first two girls were in disposables but after educating myself on the chemicals in disposables, I knew I wanted to switch to cloth.  After agonizing for hours, nay, DAYS over many reviews, blogs, cloth diapering sites, etc... I decided Charlie Banana would be the brand for us and I would not use any other diapering system.  Before my daughter was born, I used up the last

In [15]:
from transformers import BertTokenizer

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Find the longest review
# longest_review = max(loaded_data.text_edges, key=len)

# Tokenize the longest review
tokens = tokenizer(longest_review, padding=True, truncation=True, max_length=512, return_tensors="pt")


In [13]:
import torch
from transformers import BertTokenizer, BertModel

# 加载模型和分词器
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 输入文本
text = "Hello, my dog is cute"

# 将文本编码为输入ID和注意力掩码
inputs = tokenizer(longest_review, return_tensors='pt', max_length=512, truncation=True, padding=True)



In [18]:
import os
import pickle

# 指定 Dataset 目录路径
dataset_dir = "Dataset"

# 遍历 Dataset 目录中的所有数据集文件夹
for dataset_name in os.listdir(dataset_dir):
    dataset_path = os.path.join(dataset_dir, dataset_name)
    processed_dir = os.path.join(dataset_path, "processed")
    
    # 检查 processed 目录是否存在
    if os.path.isdir(processed_dir):
        print(f"Dataset: {dataset_name}")
        
        # 遍历 processed 目录中的所有文件
        for filename in os.listdir(processed_dir):
            if filename.endswith(".pkl"):
                file_path = os.path.join(processed_dir, filename)
                
                # 读取 .pkl 文件
                with open(file_path, 'rb') as file:
                    data = pickle.load(file)
                
                # 打印文件名和长度信息
                print(f"  File: {filename}")
                print(f"  Length of text_nodes: {len(data.text_nodes)}")
                print(f"  Length of text_edges: {len(data.text_edges)}")
                print("  ---")
        
        print("\n")  # 在不同数据集之间添加空行
    else:
        print(f"Warning: No 'processed' directory found in {dataset_name}")

# ... existing code ...

Dataset: goodreads_children


/home/lizhuofeng/.local/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


  File: children.pkl
  Length of text_nodes: 216613
  Length of text_edges: 734640
  ---


Dataset: goodreads_crime
  File: crime.pkl
  Length of text_nodes: 422642
  Length of text_edges: 1849236
  ---


Dataset: goodreads_history
  File: history.pkl
  Length of text_nodes: 540796
  Length of text_edges: 2066193
  ---


Dataset: reddit
  File: reddit.pkl
  Length of text_nodes: 512776
  Length of text_edges: 256388
  ---


Dataset: twitter
  File: twitter.pkl
  Length of text_nodes: 60785
  Length of text_edges: 74666
  ---


Dataset: amazon_baby
  File: baby.pkl
  Length of text_nodes: 186790
  Length of text_edges: 1241083
  ---


Dataset: goodreads_comics
  File: comics.pkl
  Length of text_nodes: 148658
  Length of text_edges: 542338
  ---


Dataset: amazon_movie
  File: movie.pkl
  Length of text_nodes: 174012
  Length of text_edges: 1697533
  ---


